# PSTAT 234 Final Project - Portfolio Optimization

### Presentation Time: Wed June 6, 11 - 12:15 pm

### Group Member:
 - Xining Li
 - Ben Ku
 - Mi Yu
 - Zhipu Zhou
 



In [12]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import math
import cvxpy as cvx
from sklearn.model_selection import KFold # import KFold to do cross validation
from sklearn.covariance import GraphLasso, GraphLassoCV, graph_lasso
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import linalg
from sklearn.preprocessing import scale

In [3]:

alpha = np.arange(0, 0.5, 0.05)
pr_all  =pd.DataFrame({'fold1':np.zeros(len(alpha)),'fold2':np.zeros(len(alpha)),
                          'fold3':np.zeros(len(alpha)),'fold4':np.zeros(len(alpha)),
                           'fold5':np.zeros(len(alpha))})

In [7]:
str('fold'+str(1))

'fold1'

In [9]:
n_fold=5
default_data ={'fold1':np.zeros(len(alpha))}

for ticker in range(n_fold):
    default_data.update({str('fold'+str(ticker+1)): np.zeros(len(alpha))})
default_data

{'fold1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'fold2': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'fold3': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'fold4': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'fold5': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

In [6]:
default_data.update({'fold2': np.zeros(len(alpha))})
default_data

{'fold1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'fold2': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

In [ ]:
{'fold1':np.zeros(len(alpha)),'fold2':np.zeros(len(alpha)),
                          'fold3':np.zeros(len(alpha)),'fold4':np.zeros(len(alpha)),
                           'fold5':np.zeros(len(alpha))}

In [4]:
pr_all

,fold1,fold2,fold3,fold4,fold5
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0


In [78]:


# //                            _ooOoo_  
# //                           o8888888o  
# //                           88" . "88  
# //                           (| -_- |)  
# //                            O\ = /O  
# //                        ____/`---'\____  
# //                      .   ' \\| |// `.  
# //                       / \\||| : |||// \  
# //                     / _||||| -:- |||||- \  
# //                       | | \\\ - /// | |  
# //                     | \_| ''\---/'' | |  
# //                      \ .-\__ `-` ___/-. /  
# //                   ___`. .' /--.--\ `. . __  
# //                ."" '< `.___\_<|>_/___.' >'"".  
# //               | | : `- \`.;`\ _ /`;.`/ - ` : | |  
# //                 \ \ `-. \_ __\ /__ _/ .-` / /  
# //         ======`-.____`-.___\_____/___.-`____.-'======  
# //                            `=---='  
# //  
# //         .............................................  
# //                  佛祖保佑             永无BUG 
# //          佛曰:  
# //                  写字楼里写字间，写字间里程序员；  
# //                  程序人员写程序，又拿程序换酒钱。  
# //                  酒醒只在网上坐，酒醉还来网下眠；  
# //                  酒醉酒醒日复日，网上网下年复年。  
# //                  但愿老死电脑间，不愿鞠躬老板前；  
# //                  奔驰宝马贵者趣，公交自行程序员。  
# //                  别人笑我忒疯癫，我笑自己命太贱；  
# //                  不见满街漂亮妹，哪个归得程序员？

import warnings
warnings.filterwarnings("ignore")


# by applying Ledoit_wolf_cv the n_fold and shrinkage_num must be specified
class portfolio_optimization():
    """This is a portfolio optimization class aiming to the Realized Return, 
    shortsize, turnover, and ending portfolio value. 
    
    Input, stock data address, selected companies name, days of period
    Using lediot_wolf, must specify n_fold and shrinkage_num
    Using graphical_lasso, must specify n_fold and alpha_num
    
    """
    def __init__(self, 
                 stock_dat_address, 
                 selected_companies_names ,
                 days_of_period,
                 start, 
                 covariance_method,
                 **method_parameters):
        self.selected_companies_names = selected_companies_names
        self.stock_data = pd.read_pickle(stock_dat_address).dropna(axis = 1)[selected_companies_names]
        self.days_of_period =days_of_period
        self.covariance_method = covariance_method
        self.start = start
        
        if covariance_method!='sample':
            
            for key, value in method_parameters.items():
                setattr(self, key, value)

    ## The optimal_solution not considering the period
     
        self.Mat_Optimization_At_Each_Period = self.Optimization_At_Each_Period()  
  
    
    
    
    


    ##This is a Lediot_wolf method's function used to do cross-validation 
    # and compare the predictive risk
    ##There might be a problem, but I cannot find it because the difference
    ##in the predictive risk seems to be pretty random. We might not use this to do
    ##cross-validation


    #input the data, the number of folds, and the number of shrinkages value we want to test
    def Ledoit_wolf_cv(self, data,n_fold,shrinkage_num):
        #The space between each fold
        z = 1.0/shrinkage_num
        shrinkage = np.arange(0,1,z)
        #create a dictionary with lambda as keys and their predictive risk as values
        #set the initial predictive risk to be 1
        shrinkage_dict = dict((el,0) for el in shrinkage)
        #loop through all the possible shrinkage values
        for l in shrinkage:
            #split into n-fold
            kf = KFold(n_splits = n_fold)
            #initialize the predictive risk
            pr_total = 0
            for train, test in kf.split(data):
                 #get the training set
                X_train = data.iloc[train]
                #calculate the sample covariance matrix
                sample = np.cov(X_train,rowvar = False)
                ##next get the Ledoit-Wolf covariance matrix using the shriankgage 
                lw = np.multiply(sample,l) + np.multiply((1.0-l),np.identity(30))
                ##calculate the omega matrix
                omega = np.linalg.inv(lw)
                ##calculate the predictive risk
                ##Also first centralize the test data
                X_test = data.iloc[test]
                X_test_central = X_test.sub(X_test.mean(axis=0), axis=1)
                X_test_central = np.asmatrix(X_test_central)
                omega_diag = np.diag(np.diag(omega))
                omega_diag_inv = np.linalg.inv(omega_diag)
                #This calculates the quantity inside the norm
                z = np.matmul(omega,omega_diag_inv)
                m = np.matmul(X_test_central,z) 
                #calculate the predictive risk
                pr = (np.linalg.norm(m,2))**2/len(test)
                pr_total += pr
            shrinkage_dict[l] = pr_total
        #This gives the best shrinkage value after the cross-validation
        best_shrinkage = min(shrinkage_dict, key=shrinkage_dict.get) 
        actual_sample = np.cov(data,rowvar = False)
        ##Using the best shrinkage to get the Ledoit-Wolf covariance matrix
        return np.multiply(actual_sample,best_shrinkage) + np.multiply((1.0-best_shrinkage),np.identity(30))

    
    
    
    
    
    
    
    
    
    




    #Graph Lasso Function
    def graphlasso_cv(self, data,alpha_num,n_fold=5):
        datac = pd.DataFrame(scale(data, axis = 0, with_mean = True, with_std = True, copy = True))
        sdv = data.std(axis = 0)
        alpha = np.arange(0, 0.5, 0.05)
        
        
        default_fold ={'fold1':np.zeros(len(alpha))}

        for ticker in range(n_fold):
            default_fold.update({str('fold'+str(ticker+1)): np.zeros(len(alpha))})
        
        pr_all  =pd.DataFrame(default_fold)
        pr_all.index = alpha
        for l in alpha:
            #split into n-fold
            kf = KFold(n_splits = n_fold)
            #initialize the predictive risk
            ii = -1
            for train, test in kf.split(datac):
                ii = ii + 1
                x_train = datac.iloc[train]
                
#                 #####################################
#                 if l==0.05:
#                     print (x_train.iloc[5,10])
#                 ######################################
                x_test = datac.iloc[test]
                ##next get the Graph Lasso covariance matrix using the shriankgage 
                model = GraphLasso(l)
                #print(x_train.shape)
                model.fit(x_train)
                omega = model.precision_
                ##calculate the predictive risk
                omega_diag = np.diag(np.diag(omega))
                omega_diag_inv = np.diag(1/np.diag(omega))
                #This calculates the quantity inside the norm
                z = np.matmul(omega,omega_diag_inv)
                m = np.matmul(x_test,z) 
                #calculate the predictive risk
                pr = (np.linalg.norm(m,2))**2/x_test.shape[0]
                pr_all.loc[l][ii] = pr
        #This gives the best alpha value after the cross-validation
        best_alpha = np.argmin(pr_all.mean(axis = 1))
        model = GraphLasso(best_alpha)
        model.fit(datac)
        omega = np.matmul(np.matmul(np.diag(sdv), model.precision_), np.diag(sdv))
        return omega, best_alpha
    
    
    

    
    
    
    def optimal_solution(self, stock_data_partition, covariance_method = 'sample', inverse = False):
        def mat_mults(*args):
            if(len(args)==1):
                return args[0]
            else:
                    return np.matmul(args[0],mat_mults(*args[1:]))
        ########

        logret = np.log(stock_data_partition).diff()
        
        if (self.covariance_method == 'sample'):
            mu = logret[1:].mean()
            sigma = logret.cov()
            s, _ = sigma.shape
            w = cvx.Variable(s)# the length of w is x
            risk = cvx.quad_form(w, sigma.as_matrix())
            prob = cvx.Problem(cvx.Minimize(risk),
                               [cvx.sum_entries(w) == 1])
            prob.solve()
            return w.value
        
        
        if (self.covariance_method == 'ledoit_wolf'):
            sigma =  self.Ledoit_wolf_cv(logret[1:],self.n_fold,self.shrinkage_num)
            ##Next let us calculate the weights
            one_vec = np.asmatrix(np.ones(len(self.selected_companies_names)))
            ##Based on the optimal solution formula
            sigma_inv = np.linalg.inv(sigma)
            one_tp = np.transpose(one_vec)
            p = np.matmul(sigma_inv,one_tp)
            q = (np.matmul(one_vec,np.matmul(sigma_inv,one_tp)))
            weight = np.multiply(1.0/float(q),p)
            return weight

        if (self.covariance_method == 'graphical_lasso'):
            sigma_inv, best_beta =  self.graphlasso_cv(logret[1:],self.alpha_num,self.n_fold)
            ##Next let us calculate the weights
            one_vec = np.asmatrix(np.ones(len(self.selected_companies_names)))
            ##Based on the optimal solution formula
            one_tp = np.transpose(one_vec)
            p = np.matmul(sigma_inv,one_tp)
            q = (np.matmul(one_vec,np.matmul(sigma_inv,one_tp)))
            weight = np.multiply(1.0/float(q),p)
            return weight



    #### Postcondition: calculate all w ##############
    def Optimization_At_Each_Period(self):
        w=[]
        for current_period_num in range(int(np.floor( (len(self.stock_data)/self.days_of_period) ))):
            if current_period_num==0:
                stock_data_partition = self.stock_data[current_period_num*self.days_of_period:(current_period_num+1)*self.days_of_period].copy()
            else:
                stock_data_partition = self.stock_data[(current_period_num*self.days_of_period-1):(current_period_num+1)*self.days_of_period].copy()
       
                
            
            
            
            w.append(self.optimal_solution(stock_data_partition))
        return np.column_stack(w)

    
    ################## Split all returns by chunk #####################
    def return_over_time(self):
        re=[]
        for current_period_num in range(int(np.floor( (len(self.stock_data)/self.days_of_period) ))):
            after = self.stock_data[(self.days_of_period*current_period_num+1):(self.days_of_period*(current_period_num+1)+1)]
            before = self.stock_data[(self.days_of_period*current_period_num):(self.days_of_period*(current_period_num+1))]
            tmp = np.log(np.array(after)/np.array(before))
            tmp = pd.DataFrame(tmp,columns = after.columns)
            re.append(tmp)

                
            

        if len(self.stock_data)/self.days_of_period-int(len(self.stock_data)/self.days_of_period)==0:
            return re
        else:
            # print ("Last Trading Period Not Finished")
            last_start=int(np.floor( (len(self.stock_data)/self.days_of_period) ))*self.days_of_period
            last_end=len(self.stock_data)-1
            before = self.stock_data[(last_start-1):(last_end-1)]
            after = self.stock_data[last_start:last_end]
            tmp = np.log(np.array(after)/np.array(before))
            tmp = pd.DataFrame(tmp,columns = after.columns)
            re.append(tmp)
            return re
        ################## Generate interval for re-balancing ####################
    def generate_interval(self):

        n = self.days_of_period
        starn = np.asscalar(np.where(self.stock_data.index == self.start)[0])

        invs = pd.Series(np.arange(starn, self.stock_data.shape[0], n))
        inve = pd.Series(np.arange(starn+n-1, self.stock_data.shape[0], n))
        inve[len(inve)] = self.stock_data.shape[0]
        este = invs - 1
        ests = invs - n
        interval = pd.DataFrame({'es':ests, 'ee':este, 'is':invs, 'ie':inve}).astype('int')
        self.interval = interval
        return interval

    ################## Calculate daily return of a portfolio #################
    def Realized_Return(self):

        interval = self.generate_interval()

        data = self.stock_data[interval.loc[0,'es']:interval.loc[interval.shape[0]-1,'ie']]
        Mat_return_over_time = self.return_over_time()  # return dataframe by chunks
        Total_time = self.Mat_Optimization_At_Each_Period.shape[1]  # Number of periods
        ss = self.shortsize()  # Size of short side
        daily_r = pd.DataFrame()
        for t in range(Total_time):
            tmp = pd.DataFrame(np.matmul(Mat_return_over_time[t + 1], self.Mat_Optimization_At_Each_Period[:,t]))
            daily_r  = pd.concat([daily_r, tmp])
            self.daily_r = daily_r
        index_num = daily_r.shape[0]
        daily_r.index =  list(self.stock_data.index)[-index_num:]
        return daily_r

    ################# Calculate short size for all periods #################
    def shortsize(self):
        ss = np.empty(self.Mat_Optimization_At_Each_Period.shape[1])  # take number of columns (number of periods)
        upper = np.empty(self.Mat_Optimization_At_Each_Period.shape[1])
        for i in range(self.Mat_Optimization_At_Each_Period.shape[1]):
            absw = np.array([np.min(np.array([np.asscalar(j),0])) for j in self.Mat_Optimization_At_Each_Period[:,i]])
            upper[i] = np.sum(np.abs(absw))
            ss[i] = upper[i] / np.sum(np.abs(self.Mat_Optimization_At_Each_Period[:,i]))
            self.upper = upper
            self.ss=ss
        return ss, upper

    ################# Calculate turnover rate for all periods ###############
    def turnover(self):
        interval = self.generate_interval()  # Interval
        Mat_return_over_time = self.return_over_time()  # return dataframe by chunks

        TO = np.empty(interval.shape[0])
        for i in range(interval.shape[0]):
            if(i == 0):
                wold = np.zeros(self.stock_data.shape[1])
            else:
                wold = np.array([j[0] for j in np.asarray(self.Mat_Optimization_At_Each_Period[:,i-1])])
            wnew = np.array([j[0] for j in np.asarray(self.Mat_Optimization_At_Each_Period[:,i])])
            TO[i] = np.sum(np.abs(wnew - np.multiply.accumulate(Mat_return_over_time[0] + 1, axis = 0).iloc[-1] * wold))
        self.TO = TO
        return TO

    ################## Calculate daily wealth of a portfolio  ################
    # borr_cost is daily rate
    def Port_Wealth(self,
                    trans_cost = 0, 
                    borr_cost = 0, 
                    init = 1):
              
        daily_r = self.Realized_Return()
        TO = self.turnover()
        ss, upper = self.shortsize()
        interval = self.generate_interval()
        
        ptf_r = daily_r.copy()
        invs = interval["is"] - interval['is'][0]
        n = interval['ee'][0] - interval['es'][0] + 1
        for k in range(interval.shape[0]):
            ptf_r.iloc[invs[k]] = ptf_r.iloc[invs[k]] - TO[k] * trans_cost - upper[k] * (math.pow(1 + borr_cost, n) - 1)
        ptf_w = init * np.multiply.accumulate(ptf_r + 1)
        index_num = ptf_w.shape[0]
        ptf_w.index =  list(self.stock_data.index)[-index_num:]
        return ptf_w

In [79]:
ticket = ['MMM','AXP','AAPL','BA','CAT','CVX','CSCO','KO','DIS','D','XOM','GE','GS','HD','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PFE','PG','TRV','UTX','UNH','VZ','V','WMT']

pos=portfolio_optimization("./closing_price_5yr.pkl",ticket, 252,'2014-05-21',covariance_method='sample')

In [50]:
def return_over_time(stock_data, days_of_period):
    re=[]
    for current_period_num in range(int(np.floor( (len(stock_data)/days_of_period) ))):
        after = stock_data[(days_of_period*current_period_num+1):(days_of_period*(current_period_num+1)+1)]
        before = stock_data[(days_of_period*current_period_num):(days_of_period*(current_period_num+1))]
        tmp = np.log(np.array(after)/np.array(before))
        tmp = pd.DataFrame(tmp,columns = after.columns)
        re.append(tmp)




    if len(stock_data)/days_of_period-int(len(stock_data)/days_of_period)==0:
        return re
    else:
        # print ("Last Trading Period Not Finished")
        last_start=int(np.floor( (len(stock_data)/days_of_period) ))*days_of_period
        last_end=len(stock_data)-1
        before = stock_data[(last_start-1):(last_end-1)]
        after = stock_data[last_start:last_end]
        tmp = np.log(np.array(after)/np.array(before))
        tmp = pd.DataFrame(tmp,columns = after.columns)
        re.append(tmp)
        return re
    


In [80]:
import pandas as pd
closing_price_data = pd.read_pickle("./closing_price_5yr.pkl").dropna(axis = 1)
closing_price_data
ticket = ['MMM','AXP','AAPL','BA','CAT','CVX','CSCO','KO','DIS','D','XOM','GE','GS','HD','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PFE','PG','TRV','UTX','UNH','VZ','V','WMT']
stock_data = closing_price_data[ticket]

r1=return_over_time(stock_data, days_of_period=252)

In [52]:
r1[2]

Name,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DIS,D,...,MSFT,NKE,PFE,PG,TRV,UTX,UNH,VZ,V,WMT
0,-0.006687,0.006049,0.008714,-0.017388,-0.007867,-0.005704,-0.005795,-0.000487,-0.001178,-0.000695,...,-0.011028,0.000288,-0.002330,-0.005737,-0.003498,-0.004206,-0.007249,-0.005830,0.003597,-0.003291
1,-0.008672,-0.013132,-0.022277,-0.013978,-0.008953,-0.015372,-0.010652,-0.005353,-0.007465,-0.006411,...,-0.006631,-0.009718,-0.004971,-0.010183,-0.011060,-0.014864,-0.011353,-0.003838,-0.015634,-0.012734
2,0.008921,-0.000998,0.018536,0.001400,0.000911,-0.001744,0.012359,0.003169,0.008462,-0.000458,...,0.021658,-0.005623,0.005554,0.003154,0.007257,0.006567,0.010181,0.003033,0.013764,0.003865
3,-0.000932,-0.000249,-0.002009,-0.004696,-0.021964,0.001260,0.001365,0.000486,0.001448,0.001973,...,-0.003367,-0.002538,0.004075,-0.000755,-0.001858,0.000425,0.001422,-0.000606,0.001008,-0.004666
4,-0.011189,-0.004631,-0.011448,-0.012799,-0.008055,-0.002327,-0.001365,-0.004384,-0.001448,-0.006361,...,-0.012512,-0.006179,0.008959,-0.011920,-0.010135,-0.004428,0.004920,-0.002022,-0.012732,-0.007647
5,-0.000629,-0.003141,0.001955,0.005182,0.002458,-0.003599,-0.004442,-0.000490,0.005331,-0.000282,...,0.007865,0.000885,-0.005482,0.005850,-0.001088,-0.001452,-0.002916,-0.004458,0.006242,0.006175
6,0.000629,0.006523,-0.004415,0.014060,0.007455,-0.000487,-0.003435,0.001220,-0.001894,-0.012992,...,-0.006585,0.003433,-0.002028,-0.003939,-0.006755,0.002902,-0.017760,-0.001628,-0.001013,-0.002680
7,0.006766,0.007596,0.001230,0.003902,0.000696,-0.004004,0.000346,-0.002930,0.003785,-0.016810,...,-0.001494,0.003911,-0.002321,0.000254,0.006755,0.009922,0.005082,-0.001220,-0.001305,0.004819
8,-0.011365,-0.015504,-0.005858,-0.018387,-0.008852,-0.007663,-0.016641,-0.006875,-0.007857,-0.008070,...,-0.010514,-0.005480,-0.004952,-0.005233,-0.007951,-0.011628,-0.014468,-0.020169,-0.010790,-0.009931
9,-0.008053,0.000377,-0.005503,-0.003263,0.006763,0.001970,-0.001048,-0.012146,0.000000,-0.010962,...,-0.004756,0.000786,-0.004390,-0.009255,-0.009022,-0.001196,-0.001030,-0.018254,0.002343,-0.014809


In [55]:
len(r1)

5

In [61]:
data = r1[2]


datac = pd.DataFrame(scale(data, axis = 0, with_mean = True, with_std = True, copy = True))
sdv = data.std(axis = 0)
alpha = np.arange(0, 0.5, 0.05)



n_fold=5
default_fold ={'fold1':np.zeros(len(alpha))}

for ticker in range(n_fold):
    default_fold.update({str('fold'+str(ticker+1)): np.zeros(len(alpha))})

pr_all  =pd.DataFrame(default_fold)
pr_all.index = alpha
for l in alpha:
    #split into n-fold
    print(l)
    kf = KFold(n_splits = n_fold)
    #initialize the predictive risk
    ii = -1
    for train, test in kf.split(datac):
        ii = ii + 1
        x_train = datac.iloc[train]
        x_test = datac.iloc[test]
        

        #####################################
        if l==0.05:
            print (x_train.iloc[5,10])
        ######################################
        
        
        
        ##next get the Graph Lasso covariance matrix using the shriankgage 
        model = GraphLasso(l)
        #print(x_train.shape)
        model.fit(x_train)
        omega = model.precision_
        ##calculate the predictive risk
        omega_diag = np.diag(np.diag(omega))
        omega_diag_inv = np.diag(1/np.diag(omega))
        #This calculates the quantity inside the norm
        z = np.matmul(omega,omega_diag_inv)
        m = np.matmul(x_test,z) 
        #calculate the predictive risk
        pr = (np.linalg.norm(m,2))**2/x_test.shape[0]
        pr_all.loc[l][ii] = pr
#This gives the best alpha value after the cross-validation
best_alpha = np.argmin(pr_all.mean(axis = 1))
model = GraphLasso(best_alpha)
model.fit(datac)
omega = np.matmul(np.matmul(np.diag(sdv), model.precision_), np.diag(sdv))
omega

0.0
0.05
1.0800286972462982
-0.0709241315696723
-0.0709241315696723
-0.0709241315696723
-0.0709241315696723
0.1
0.15000000000000002
0.2
0.25
0.30000000000000004
0.35000000000000003
0.4
0.45


array([[ 2.66592833e-04,  0.00000000e+00, -8.91788509e-06,
        -1.01280855e-06, -3.41711444e-05,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -3.56736204e-07,
         0.00000000e+00, -1.08224116e-05, -5.44016719e-05,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.87955664e-05, -1.60446074e-05,
        -1.41114869e-05,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.50837461e-05,
        -2.07593983e-05, -3.32511539e-05,  0.00000000e+00,
        -2.39418677e-05,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  2.91782859e-04,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -3.64410157e-05,
        -1.27248846e-05,  0.00000000e+00, -1.87734902e-05,
         0.00000000e+00,  0.00000000e+00, -2.92248236e-05,
         0.00000000e+00, -2.99450017e-06,  0.00000000e+

In [81]:
ticket = ['MMM','AXP','AAPL','BA','CAT','CVX','CSCO','KO','DIS','D','XOM','GE','GS','HD','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PFE','PG','TRV','UTX','UNH','VZ','V','WMT']

po2=portfolio_optimization("./closing_price_5yr.pkl",ticket, 252,'2014-05-21',covariance_method='graphical_lasso',alpha_num=10,n_fold=5)

In [82]:
po2.Port_Wealth()

,0
2014-05-22,1.001109
2014-05-23,1.007649
2014-05-27,1.016888
2014-05-28,1.014596
2014-05-29,1.022666
2014-05-30,1.025312
2014-06-02,1.024471
2014-06-03,1.023254
2014-06-04,1.025506
2014-06-05,1.033205


In [76]:
ticket = ['MMM','AXP','AAPL','BA','CAT','CVX','CSCO','KO','DIS','D','XOM','GE','GS','HD','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PFE','PG','TRV','UTX','UNH','VZ','V','WMT']

po2=portfolio_optimization("./closing_price_5yr.pkl",ticket, 252,'2014-05-21',covariance_method='ledoit_wolf',shrinkage_num=10,n_fold=5)

In [77]:
po2.Port_Wealth()

,0
2014-05-22,1.001328
2014-05-23,1.004484
2014-05-27,1.009375
2014-05-28,1.007414
2014-05-29,1.012142
2014-05-30,1.014063
2014-06-02,1.015602
2014-06-03,1.015263
2014-06-04,1.016197
2014-06-05,1.022126


In [160]:
def portfolio_value(n):
    tmp = portfolio_optimization("./closing_price_5yr.pkl",ticket, n,'2014-05-21')
    if (n<tmp.stock_data.shape[1]):
        return np.nan
    else:
        return tmp.Port_Wealth().iloc[-1:].values[0][0]

In [162]:
list(map(portfolio_value,[21,63,126,262]))

[nan, 0.9069856230798227, 1.4779937991942707, 1.2219045269130393]

In [163]:
po.Mat_Optimization_At_Each_Period

matrix([[ 0.12175338, -0.15350017,  0.03202763, ..., -0.15119153,
         -0.24572279, -0.63837422],
        [-0.13092679,  0.13115644, -0.12896231, ..., -0.58849685,
         -0.54366081, -0.1362559 ],
        [ 0.48487168,  0.21159928,  0.02470386, ..., -0.08719473,
         -0.4853602 , -0.25892601],
        ...,
        [ 0.10661925, -0.16541563,  0.04592203, ..., -0.14880982,
          0.13409742, -0.24691324],
        [-0.15956028, -0.47217194,  0.03172527, ...,  0.03218914,
          0.23495587,  0.13058959],
        [ 0.18916849,  0.06144344,  0.05197028, ..., -0.11184307,
          0.32706739, -0.18653387]])

In [139]:
po

In [129]:
a.shape

(1237, 1)

In [138]:
a.iloc[-1:].values[0][0]

1.812886590655414

In [108]:
po.Port_Wealth

,0
2013-06-21,0.000562
2013-06-24,-0.009072
2013-06-25,0.003396
2013-06-26,0.005182
2013-06-27,0.003667
2013-06-28,0.004603
2013-07-01,0.005177
2013-07-02,-0.007564
2013-07-03,0.001480
2013-07-05,0.008006


In [96]:
a.shape[0]

1237

In [87]:
po.stock_data.shape

(1259, 467)

In [93]:
a.index = list(po.stock_data.index)[-1237:]

In [94]:
a

,0
2013-06-21,1.000562
2013-06-24,0.991485
2013-06-25,0.994853
2013-06-26,1.000008
2013-06-27,1.003675
2013-06-28,1.008295
2013-07-01,1.013515
2013-07-02,1.005849
2013-07-03,1.007337
2013-07-05,1.015402


In [84]:
po.Realized_Return()

,0
0,0.000562
1,-0.009072
2,0.003396
3,0.005182
4,0.003667
5,0.004603
6,0.005177
7,-0.007564
8,0.001480
9,0.008006
